In [1]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time
import datetime

import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from scipy.signal import savgol_filter
import statsmodels.formula.api as smf

In [2]:
frida_products = get_fresh_query_result("https://internal-redash.federate.frubana.com/",118629,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [3]:
df_check = get_fresh_query_result("https://internal-redash.federate.frubana.com/",165821,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [ ]:
#df_margin = get_fresh_query_result("https://internal-redash.federate.frubana.com/",144521,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

In [4]:
df_margin = df_check.loc[~(df_check.fit_for_discount == 'FALSE'),['card_id','margin']].groupby(['card_id']).margin.mean().reset_index()

In [5]:
custom = pd.read_csv("push customers - customers.csv")

In [125]:
print(f"HOLA {city}")

HOLA CWB


In [96]:
city = 'CWB'
clientes = custom.loc[custom.city == city].customer_id.unique().tolist()
dias = 0
#year-month-day
todays_date = datetime.datetime.today().strftime('%Y-%m-%d')
#non_buyers_date = '2022-08-26' #1 months
customer_date_desde_query = (datetime.datetime.today() - datetime.timedelta(days=30)).strftime('%Y-%m-%d') #1 months

customer_date_desde = (datetime.datetime.today() - datetime.timedelta(days=200))#.strftime('%Y-%m-%d') #1 months

In [97]:
custom

,customer_id,city
0,360087971,BHZ
1,156245729,BHZ
2,253061474,BHZ
3,358570122,BHZ
4,96943903,BHZ
...,...,...
15571,370678031,VCP
15572,362038761,VCP
15573,203463056,VCP
15574,222861546,VCP


In [98]:
len(clientes)

1307

In [99]:
query = """
-- 93058
-- Se toma como base el query 2718
WITH 
ful AS (

SELECT DISTINCT
    bo.order_id,
    bo.submit_date
    
FROM postgres_broadleaf_federate."broadleaf.blc_order" bo
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group" bfg        ON bfg.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order" bfo        ON bfo.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group" ffg         ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order" fo                      ON fo.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_payment" bop            ON bop.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site" s                       ON s.site_id = bo.site_disc
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type" fot                ON fot.fb_order_type_id=fo.fb_order_type_id

WHERE fo.fb_order_status_id IN (1,6,7,8)
    AND extract(year from bo.submit_date::date) > 2021
    --AND extract(month from bo.submit_date::date) = 6--2021
    AND bo.order_status = 'SUBMITTED'
    AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
    AND bop.archived = 'N'
    AND (fot.name IS NULL OR fot.name <> 'REFUND')
    AND s.site_identifier_value = '{city}' --NOT IN ('MTY','PBC','GDL')
    AND bo.customer_id IN {clientes}
)--,

--more AS (
SELECT DISTINCT
    s.site_identifier_value AS city,
    bo.customer_id,
    --DATE(bo.submit_date) AS submit_date,
    bs.addl_product_id AS card_id,
    bs2.name,
    COUNT(DISTINCT bo.order_id) AS frequency,
    SUM(boipd.quantity * foi.step_unit * boi.sale_price)*1.0 AS gmv_usd,
    SUM(boipd.quantity*foi.step_unit) AS demand

FROM postgres_broadleaf_federate."broadleaf.blc_order" bo
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                     s       ON s.site_id = bo.site_disc
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"               boi     ON boi.order_id=bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_item"                foi     ON boi.order_item_id= foi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                     fo      ON fo.order_id = bo.order_id
INNER JOIN ful                                                                          ON ful.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_discrete_order_item"      bdoi    ON bdoi.order_item_id = boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                      bs      ON bs.sku_id = bdoi.sku_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"                  bp      ON bs.addl_product_id = bp.product_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                      bs2     ON bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                 bcat    ON bcat.category_id = bp.default_category_id 
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category_xref"            bcx     ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                 bcat2   ON bcx.category_id = bcat2.category_id
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_price_dtl"      boipd   ON boipd.order_item_id=boi.order_item_id
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_dtl_adj"        boida   ON boida.order_item_price_dtl_id=boipd.order_item_price_dtl_id

WHERE fb_order_status_id IN (1,6,7,8)
    AND bo.order_status = 'SUBMITTED'
   -- FIX SUPER DESCUENTOS
    AND bcat2.category_id not in ('110873','-1000','100768','100765','100815') --ids de super descuentos en cada país
    AND bcat.name <> 'Oferton Frubana' AND bcat2.name <> 'Oferton Frubana'
GROUP BY 1,2,3,4
""".format(clientes=tuple(set(clientes)), city = city)

data_ventas_aux = read_connection_data_warehouse.runQuery(query)
data_ventas_aux[["frequency", "gmv_usd","demand"]] = data_ventas_aux[["frequency", "gmv_usd","demand"]].astype('float32')
#data_ventas_aux['submit_date'] = pd.to_datetime(data_ventas_aux['submit_date']) 

In [100]:
# Assuming your DataFrames are named 'df1' and 'df2'
data_ventas_aux = pd.merge(data_ventas_aux, df_margin[['margin','card_id']], left_on='card_id', right_on='card_id', how='inner')

In [101]:
data_ventas_aux = data_ventas_aux.loc[~data_ventas_aux.card_id.isin([55148,80182,-201566,-201623,748984,749057,749205,749278,749352,749426])].reset_index(drop=True)

In [102]:
# Optional to clean the EagleEye
print(len(data_ventas_aux))

18815


In [103]:
data_ventas_aux['cash_margin'] = data_ventas_aux.gmv_usd * data_ventas_aux.margin

In [104]:
# Assuming your DataFrame is named 'df'
grouped_df = data_ventas_aux.groupby('customer_id').apply(lambda x: x.nlargest(7, ['frequency', 'gmv_usd'])).reset_index(drop=True)

result_df = grouped_df.groupby('customer_id').agg({'card_id': list, 'name': list, 'frequency': list, 'gmv_usd': np.sum, 'cash_margin': np.sum}).reset_index()

result_df["margin"] = 100.00 * result_df.cash_margin / result_df.gmv_usd
result_df['margin'] = np.where(0.80*result_df.margin > 13, 13,0.80*result_df.margin)
result_df['margin'] = result_df['margin'].apply(lambda x: round(x, 2))
result_df['offer_name'] = ["SH_GRADS_"+city+"_"+todays_date+"_"+str(customer) for customer in result_df.customer_id]

result_df = result_df.loc[result_df.margin > 3].copy()

In [105]:
# Step 1: Convert 'card_id' column from lists to individual values
df_exploded = result_df.explode('card_id')

# Step 2: Count the occurrences of each card ID
card_id_counts = df_exploded['card_id'].value_counts().reset_index()

# Rename columns and set 'card_id' as the index
card_id_counts.rename(columns={'index': 'card_id', 'card_id': 'frequency'}, inplace=True)

# Calculate cumulative percentage and percentage over the total
card_id_counts['cumulative_percentage'] = card_id_counts['frequency'].cumsum() / card_id_counts['frequency'].sum()
card_id_counts['percentage_over_total'] = card_id_counts['frequency'] / card_id_counts['frequency'].sum()

# Multiply by 100 and round to two decimal places
card_id_counts['cumulative_percentage'] = (card_id_counts['cumulative_percentage'] * 100).round(2)
card_id_counts['percentage_over_total'] = (card_id_counts['percentage_over_total'] * 100).round(2)

In [106]:
df_demand =  pd.merge(card_id_counts, data_ventas_aux[['card_id','name']], left_on='card_id', right_on='card_id', how='inner')
df_demand.drop_duplicates(subset=['card_id'], keep='first', inplace=True)

In [107]:
def concatenate_skus(card_id_list):
    result = []
    for sku in card_id_list:
        try:
            sku_value = frida_products.loc[frida_products['id_tarjeta'] == sku, 'skus'].values[0]
            result.append(sku_value)
        except IndexError:
            result.append('')
    return '#'.join(result)

In [108]:
from sklearn.model_selection import train_test_split 

def create_groups(offer):
    
    #customers = offer.customer_id.unique()
    customers = offer.customer_id.unique()
    print("Total length: ", len(customers))
    customer_control, customer_test = train_test_split(customers, test_size = 0.50)
    
    alpha=0.01
    aux=0
    while abs(1-(offer[offer.customer_id.isin(customer_control)]['gmv_usd'].mean()/offer[offer.customer_id.isin(customer_test)]['gmv_usd'].mean())) > alpha:
        customer_control, customer_test = train_test_split(customers, test_size = 0.50)
        
        aux+=1
        if aux == 1000:
            alpha+=0.01
            aux=0
        
    return customer_control.tolist(), customer_test.tolist()

In [109]:
control,test = create_groups(result_df)

Total length:  1281


In [110]:
final_df = pd.concat([
    pd.DataFrame({'group': 'control', 'customer_id': control}),
    pd.DataFrame({'group': 'test', 'customer_id': test})
], ignore_index=True)

final_df['city'] = city
final_df['todays_date'] = todays_date

In [111]:
final_df.group.value_counts()

test       641
control    640
Name: group, dtype: int64

In [112]:
final_df.to_csv("SH_GRADS"+todays_date+city+"GROUPS_9a16.csv", index=False)

In [113]:
global_segment_list = []

for i in result_df.loc[result_df.customer_id.isin(test)].values:
    global_segment_list.append({'name':i[-1], 'customersIds': [i[0]],"expiresAt": (datetime.datetime.today() + datetime.timedelta(days=(dias+2))).strftime('%Y-%m-%d')})
    
result_df = result_df.loc[result_df.customer_id.isin(test)].reset_index(drop=True).copy()

In [114]:
csv_file = {} 
csv_file['offer_name'] = result_df.offer_name
csv_file['offer_description'] = None
csv_file['discount'] = result_df.margin
csv_file['automatically_consider_offer'] = "true"
csv_file['start_date'] = (datetime.datetime.today() + datetime.timedelta(days=dias)).strftime('%d-%m-%Y')
csv_file['end_date'] = (datetime.datetime.today() + datetime.timedelta(days=(dias+2))).strftime('%d-%m-%Y')
csv_file['max_uses_per_order'] = 120
csv_file['max_uses_per_customer'] = 2
csv_file['customer_segment_id'] = [1]*len(result_df.offer_name) #[None]
csv_file['sku_id'] = result_df['card_id'].apply(concatenate_skus)
csv_file['segment_behaviour'] = 'INCLUDED'
csv_file['offer_type'] = 'SUPER_HOOK'
csv_file['supplier_discount'] = 0

global_offer_csv = pd.DataFrame(csv_file) #, index=[0])

In [115]:
global_offer_csv.head()

,offer_name,offer_description,discount,automatically_consider_offer,start_date,end_date,max_uses_per_order,max_uses_per_customer,customer_segment_id,sku_id,segment_behaviour,offer_type,supplier_discount
0,SH_GRADS_CWB_2024-03-20_110371528,None,10.87,true,20-03-2024,22-03-2024,120,2,1,363165#578143#363213#391139#385404#637967#453384,INCLUDED,SUPER_HOOK,0
1,SH_GRADS_CWB_2024-03-20_110530410,None,13.00,true,20-03-2024,22-03-2024,120,2,1,369891#649881#368392#363179#583749#363203#3631...,INCLUDED,SUPER_HOOK,0
2,SH_GRADS_CWB_2024-03-20_110587542,None,7.56,true,20-03-2024,22-03-2024,120,2,1,#370809#370856#421313#370915#391139,INCLUDED,SUPER_HOOK,0
3,SH_GRADS_CWB_2024-03-20_110974782,None,12.68,true,20-03-2024,22-03-2024,120,2,1,623189#385443#371317#363165#477628#363213#363253,INCLUDED,SUPER_HOOK,0
4,SH_GRADS_CWB_2024-03-20_111735559,None,13.00,true,20-03-2024,22-03-2024,120,2,1,363304#363195#363141#363203#424298#363273#645649,INCLUDED,SUPER_HOOK,0


In [116]:
# Replace two or three '#' with one '#'
global_offer_csv['sku_id'] = global_offer_csv['sku_id'].str.replace('###', '#')
global_offer_csv['sku_id'] = global_offer_csv['sku_id'].str.replace('##', '#')
    
# Replace '#' at the beginning with an empty string
global_offer_csv['sku_id'] = global_offer_csv['sku_id'].str.replace('^#', '', regex=True)

# Remove rows where the string is empty
global_offer_csv = global_offer_csv.loc[global_offer_csv['sku_id'] != ""]

In [117]:
global_offer_csv

,offer_name,offer_description,discount,automatically_consider_offer,start_date,end_date,max_uses_per_order,max_uses_per_customer,customer_segment_id,sku_id,segment_behaviour,offer_type,supplier_discount
0,SH_GRADS_CWB_2024-03-20_110371528,None,10.87,true,20-03-2024,22-03-2024,120,2,1,363165#578143#363213#391139#385404#637967#453384,INCLUDED,SUPER_HOOK,0
1,SH_GRADS_CWB_2024-03-20_110530410,None,13.00,true,20-03-2024,22-03-2024,120,2,1,369891#649881#368392#363179#583749#363203#3631...,INCLUDED,SUPER_HOOK,0
2,SH_GRADS_CWB_2024-03-20_110587542,None,7.56,true,20-03-2024,22-03-2024,120,2,1,370809#370856#421313#370915#391139,INCLUDED,SUPER_HOOK,0
3,SH_GRADS_CWB_2024-03-20_110974782,None,12.68,true,20-03-2024,22-03-2024,120,2,1,623189#385443#371317#363165#477628#363213#363253,INCLUDED,SUPER_HOOK,0
4,SH_GRADS_CWB_2024-03-20_111735559,None,13.00,true,20-03-2024,22-03-2024,120,2,1,363304#363195#363141#363203#424298#363273#645649,INCLUDED,SUPER_HOOK,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
636,SH_GRADS_CWB_2024-03-20_370446372,None,13.00,true,20-03-2024,22-03-2024,120,2,1,363171#363257#363203#363201#363273#368422#3684...,INCLUDED,SUPER_HOOK,0
637,SH_GRADS_CWB_2024-03-20_370515802,None,13.00,true,20-03-2024,22-03-2024,120,2,1,423482#652055#385404#385443#385434#377327#369513,INCLUDED,SUPER_HOOK,0
638,SH_GRADS_CWB_2024-03-20_370538450,None,7.23,true,20-03-2024,22-03-2024,120,2,1,385404#381444,INCLUDED,SUPER_HOOK,0
639,SH_GRADS_CWB_2024-03-20_370645887,None,7.93,true,20-03-2024,22-03-2024,120,2,1,363177#368404#385404#381444#621867,INCLUDED,SUPER_HOOK,0


In [118]:
import requests
import json

url = f"https://{city}.frubana.com/api/v1/segments/create?api-key=00_growth-team-zzz-qqq_001"

payload = json.dumps(global_segment_list)


headers = {
  'Cookie': 'AWSALB=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALBCORS=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALB=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI; AWSALBCORS=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

In [119]:
response.text

'Customer segments were created: [653457-SH_GRADS_CWB_2024-03-20_110371528, 653458-SH_GRADS_CWB_2024-03-20_110530410, 653461-SH_GRADS_CWB_2024-03-20_110587542, 653462-SH_GRADS_CWB_2024-03-20_110974782, 653460-SH_GRADS_CWB_2024-03-20_111735559, 653459-SH_GRADS_CWB_2024-03-20_113396816, 653463-SH_GRADS_CWB_2024-03-20_113663757, 653465-SH_GRADS_CWB_2024-03-20_114941171, 653464-SH_GRADS_CWB_2024-03-20_115368494, 653466-SH_GRADS_CWB_2024-03-20_115404142, 653467-SH_GRADS_CWB_2024-03-20_115473103, 653468-SH_GRADS_CWB_2024-03-20_116025601, 653469-SH_GRADS_CWB_2024-03-20_116079645, 653470-SH_GRADS_CWB_2024-03-20_116259926, 653471-SH_GRADS_CWB_2024-03-20_116419102, 653472-SH_GRADS_CWB_2024-03-20_116551655, 653473-SH_GRADS_CWB_2024-03-20_116572513, 653474-SH_GRADS_CWB_2024-03-20_117440035, 653475-SH_GRADS_CWB_2024-03-20_117462694, 653476-SH_GRADS_CWB_2024-03-20_117860309, 653478-SH_GRADS_CWB_2024-03-20_117915499, 653479-SH_GRADS_CWB_2024-03-20_118464040, 653477-SH_GRADS_CWB_2024-03-20_118478733, 

In [120]:
sgt = [(int(x[:6]),x[7:]) for x in list(map(str.strip, response.text[32:].strip('][').replace('"', '').split(',')))]

In [121]:
for i in global_offer_csv.offer_name:
    for j,z in sgt:
        if z==i:
            global_offer_csv.loc[global_offer_csv.offer_name == i, ('customer_segment_id')] = j
            break

In [122]:
global_offer_csv.head()

,offer_name,offer_description,discount,automatically_consider_offer,start_date,end_date,max_uses_per_order,max_uses_per_customer,customer_segment_id,sku_id,segment_behaviour,offer_type,supplier_discount
0,SH_GRADS_CWB_2024-03-20_110371528,None,10.87,true,20-03-2024,22-03-2024,120,2,653457,363165#578143#363213#391139#385404#637967#453384,INCLUDED,SUPER_HOOK,0
1,SH_GRADS_CWB_2024-03-20_110530410,None,13.00,true,20-03-2024,22-03-2024,120,2,653458,369891#649881#368392#363179#583749#363203#3631...,INCLUDED,SUPER_HOOK,0
2,SH_GRADS_CWB_2024-03-20_110587542,None,7.56,true,20-03-2024,22-03-2024,120,2,653461,370809#370856#421313#370915#391139,INCLUDED,SUPER_HOOK,0
3,SH_GRADS_CWB_2024-03-20_110974782,None,12.68,true,20-03-2024,22-03-2024,120,2,653462,623189#385443#371317#363165#477628#363213#363253,INCLUDED,SUPER_HOOK,0
4,SH_GRADS_CWB_2024-03-20_111735559,None,13.00,true,20-03-2024,22-03-2024,120,2,653460,363304#363195#363141#363203#424298#363273#645649,INCLUDED,SUPER_HOOK,0


In [123]:
#//--------------------------------------------
#//Exporting single file data
global_offer_csv.to_csv(f'{todays_date}_SH_FARMING_{city}_OFFER_csv.csv', index = False)

In [124]:
#result_df.to_excel(f'SD/export_data/Below_Media/{city}/{todays_date}_SH_RESULTS_FARMING_{city}_OFFER_csv.xlsx', index = False)